In [1]:
import re
import pandas as pd
import os

In [2]:
#caso particolare perché sono un coglione e non ho considerato il fatto che ho tini diversi
def findTini(file):
    if "55627" in file:
        return "Tini-2"
    elif "55571" in file:
        return "Tini-1"
    elif "55496" in file:
        return "Tini0"


In [3]:
def format_data_name(data_name,file):
    # Estrai le parti del percorso del file
    parts = data_name.split('/')
    # Trova il nome del file senza estensione
    base_name = parts[-1].replace('.dat', '')
    # Trova la sottocartella (AP o NC)
    subfolder = parts[-2] if parts[-2] in ['AP', 'NC'] else ''
    # Trova il tipo di file (grid, airport, metroplex)
    if 'grid' in base_name:
        file_type = 'grid'
    elif 'airport' in base_name:
        file_type = 'airport'
    elif 'metroplex' in base_name:
        file_type = 'metroplex'
    else:
        file_type = ''

    # Aggiungi un'indicazione speciale se contiene "fixed"
    fixed = 'Fixed' if 'fixed' in data_name.lower() else ''

    # Componi il nome nel formato desiderato
    if subfolder:
        tini = findTini(file)
        formatted_name = f"{file_type}{subfolder}{fixed}{base_name[len(file_type):]}{tini}"
    else:
        formatted_name = f"{base_name}{fixed}"

    return formatted_name

In [4]:
def parse_variable_counts(text,file):
    # Pattern per catturare tutte le occorrenze di data_name
    data_name_pattern = re.compile(r"data_name\s*=\s*['\"](.+?)['\"]", re.DOTALL)
    variable_pattern = re.compile(r"(\d+)\s*variables:.*?(\d+)\s*binary\s*variables.*?(\d+)\s*integer\s*variables.*?(\d+)\s*linear\s*variables", re.DOTALL)
    data_names = data_name_pattern.findall(text)
    variable_matches = variable_pattern.findall(text)

    # Assumiamo che i blocchi di dati siano nello stesso ordine nei log
    data = []
    if (len(data_names) != len(variable_matches)):
        print ("Error")
        raise Exception("Mismatch between data names and variable counts")
    else:
        min_length = min(len(data_names), len(variable_matches))  # Evita mismatch

        for i in range(min_length):
            data_name = data_names[i]
            total_vars, binary_vars, integer_vars, linear_vars = map(int, variable_matches[i])
            formatted_name = format_data_name(data_name,file)
            data.append([formatted_name, total_vars, binary_vars, integer_vars, linear_vars])
        return data

In [5]:
# Percorso della cartella contenente i file di log
folder_path = "/home/magi/UAMdeconflictionMasterThesis/modelli/out/pelegrin"  # Sostituiscilo con il percorso corretto
prefix = "ampl_output"  # Modifica il prefisso in base alle tue esigenze

# Ottenere tutti i file che iniziano con il prefisso specificato
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.startswith(prefix) and os.path.isfile(os.path.join(folder_path, f))]

# Lista per raccogliere i dati da tutti i file
all_data = []

for file in files:
    with open(file, "r") as f:
        content = f.read()
        all_data.extend(parse_variable_counts(content,file))

# Creazione del DataFrame
df = pd.DataFrame(all_data, columns=["Data Name", "Total", "Binary", "Integer", "Linear"])

# Filtra i dati per quelli che contengono "Fixed" e quelli che non lo contengono
df_fixed = df[df["Data Name"].str.contains("Fixed")]
df_non_fixed = df[~df["Data Name"].str.contains("Fixed")]

df_fixed.loc[:, "Data Name"] = df_fixed["Data Name"].str.replace("Fixed", "", regex=False)
# Raggruppamento e somma delle variabili per Data Name
df_fixed_grouped = df_fixed.groupby("Data Name").sum().reset_index()
df_non_fixed_grouped = df_non_fixed.groupby("Data Name").sum().reset_index()



In [23]:
# Converti i DataFrame in tabelle LaTeX
latex_table_fixed = df_fixed_grouped.to_latex("/home/magi/UAMdeconflictionMasterThesis/risultati/mercedes/fixed_table.tex",longtable=True,index=False)
latex_table_non_fixed = df_non_fixed_grouped.to_latex("/home/magi/UAMdeconflictionMasterThesis/risultati/mercedes/non_fixed_table.tex",longtable=True,index=False)

# salva i csv
df_fixed_grouped.to_csv("/home/magi/UAMdeconflictionMasterThesis/risultati/mercedes/fixed_table.csv", index=False)
df_non_fixed_grouped.to_csv("/home/magi/UAMdeconflictionMasterThesis/risultati/mercedes/non_fixed_table.csv", index=False)

### Drift and Delay

In [7]:
temp_df = df[df["Data Name"].str.contains(r"airport\d+nDr0nDe1")]
df_air01F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_air01NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"airport\d+nDr0nDe2")]
df_air02F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_air02NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"airport\d+nDr0nDe3")]
df_air03F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_air03NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"airport\d+nDr1nDe0")]
df_air10F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_air10NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"airport\d+nDr1nDe1")]
df_air11F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_air11NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]



In [8]:
temp_df = df[df["Data Name"].str.contains(r"grid\d+nDr0nDe1")]
df_grid01F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_grid01NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"grid\d+nDr0nDe2")]
df_grid02F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_grid02NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"grid\d+nDr0nDe3")]
df_grid03F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_grid03NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"grid\d+nDr1nDe0")]
df_grid10F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_grid10NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"grid\d+nDr1nDe1")]
df_grid11F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_grid11NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]



In [9]:
temp_df = df[df["Data Name"].str.contains(r"metroplex\d+nDr0nDe1")]
df_mp01F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mp01NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"metroplex\d+nDr0nDe2")]
df_mp02F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mp02NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"metroplex\d+nDr0nDe3")]
df_mp03F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mp03NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"metroplex\d+nDr1nDe0")]
df_mp10F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mp10NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = df[df["Data Name"].str.contains(r"metroplex\d+nDr1nDe1")]
df_mp11F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mp11NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]



### AP

In [10]:
airport_df= df[df["Data Name"].str.contains(r"airportAP")]
temp_df = airport_df[airport_df["Data Name"].str.contains(r"Tini0")]
df_airAP0F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_airAP0NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = airport_df[airport_df["Data Name"].str.contains(r"Tini-1")]
df_airAP1F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_airAP1NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = airport_df[airport_df["Data Name"].str.contains(r"Tini-2")]
df_airAP2F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_airAP12F = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

In [11]:
grid_df = df[df["Data Name"].str.contains(r"gridAP")]
temp_df = grid_df[grid_df["Data Name"].str.contains(r"Tini0")]
df_gridAP0F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_gridAP0NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = grid_df[grid_df["Data Name"].str.contains(r"Tini-1")]
df_gridAP1F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_gridAP1NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = grid_df[grid_df["Data Name"].str.contains(r"Tini-2")]
df_gridAP2F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_gridAP2NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

In [12]:
metroplex_df = df[df["Data Name"].str.contains(r"metroplexAP")]
temp_df = metroplex_df[metroplex_df["Data Name"].str.contains(r"Tini0")]
df_mpAP0F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mpAP0NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = metroplex_df[metroplex_df["Data Name"].str.contains(r"Tini-1")]
df_mpAP1F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mpAP1NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = metroplex_df[metroplex_df["Data Name"].str.contains( r"Tini-2")]
df_mpAP2F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mpAP2NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]
temp_df

,Data Name,Total,Binary,Integer,Linear
52,metroplexAP33Tini-2,25964,17759,95,6197
54,metroplexAPFixed33Tini-2,20832,15258,95,5479
74,metroplexAP69Tini-2,19884,14456,93,5335
76,metroplexAPFixed69Tini-2,19884,14456,93,5335
128,metroplexAP79Tini-2,24785,16891,90,5914
...,...,...,...,...,...
6322,metroplexAPFixed41Tini-2,21710,15934,94,5682
6386,metroplexAP54Tini-2,25192,17740,98,6248
6388,metroplexAPFixed54Tini-2,21882,16028,98,5756
6392,metroplexAP6Tini-2,18272,13256,86,4930


### NC

In [13]:
airport_df = df[df["Data Name"].str.contains(r"airportNC")]

temp_df = airport_df[airport_df["Data Name"].str.contains(r"Tini0")]
df_airNC0F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_airNC0NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = airport_df[airport_df["Data Name"].str.contains(r"Tini-1")]
df_airNC1F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_airNC1NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = airport_df[airport_df["Data Name"].str.contains(r"Tini-2")]
df_airNC2F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_airNC2NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

In [14]:
grid_df = df[df["Data Name"].str.contains(r"gridNC")]

temp_df = grid_df[grid_df["Data Name"].str.contains(r"Tini0")]
df_gridNC0F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_gridNC0NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = grid_df[grid_df["Data Name"].str.contains(r"Tini-1")]
df_gridNC1F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_gridNC1NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = grid_df[grid_df["Data Name"].str.contains(r"Tini-2")]
df_gridNC2F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_gridNC2NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

In [15]:
metroplex_df= df[df["Data Name"].str.contains(r"metroplexNC")]

temp_df = metroplex_df[metroplex_df["Data Name"].str.contains(r"Tini0")]
df_mpNC0F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mpNC0NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = metroplex_df[metroplex_df["Data Name"].str.contains(r"Tini-1")]
df_mpNC1F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mpNC1NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

temp_df = metroplex_df[metroplex_df["Data Name"].str.contains(r"Tini-2")]
df_mpNC2F = temp_df[temp_df["Data Name"].str.contains("Fixed")]
df_mpNC2NF = temp_df[~temp_df["Data Name"].str.contains("Fixed")]

In [16]:
def get_formatted_name(data_name):
    data_name = data_name.replace("Fixed", "")
    match = re.match(r"([a-zA-Z]+)(\d+)([a-zA-Z0-9-]+)", data_name)
    if match:
        part1 = match.group(1)
        part2 = match.group(3)
        return f"{part1}{part2}"
    else:
        return data_name  # Caso in cui non ci sia un numero nel nome

In [22]:
df_mpNC1NF

,Data Name,Total,Binary,Integer,Linear
13,metroplexNC98Tini-1,24047,15398,85,5690
45,metroplexNC40Tini-1,10391,6523,57,3497
107,metroplexNC64Tini-1,11573,7336,64,3889
115,metroplexNC89Tini-1,17952,11139,69,4770
135,metroplexNC41Tini-1,26491,18331,91,6136
...,...,...,...,...,...
6161,metroplexNC47Tini-1,8973,5852,53,3068
6227,metroplexNC28Tini-1,14966,7864,55,4130
6233,metroplexNC44Tini-1,15489,9816,70,4437
6343,metroplexNC30Tini-1,25732,18554,95,5932


### latex Creation

In [18]:
dfsF = [df_air11F,df_air10F,df_air01F,df_air02F,df_air03F,df_grid11F,df_grid10F,df_grid01F,df_grid02F,df_grid03F,df_mp11F,df_mp10F,df_mp01F,df_mp02F,df_mp03F]
dfsF += [df_airAP0F,df_airAP1F,df_airAP2F,df_gridAP0F,df_gridAP1F,df_gridAP2F,df_mpAP0F,df_mpAP1F,df_mpAP2F,df_airNC0F,df_airNC1F,df_airNC2F,df_gridNC0F,df_gridNC1F,df_gridNC2F,df_mpNC0F,df_mpNC1F,df_mpNC2F]

with open("/home/magi/UAMdeconflictionMasterThesis/risultati/mercedes/SummaryTablesFixed.tex", "w") as f:
    f.write("\\begin{document}\n")
    #prima tabella: numero di variabili totale
    results = []
    for i, df in enumerate(dfsF):
        column = df["Total"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of variable of Mercedes instances with all path fixed", label="table:mercedes:totalVar:fixed", longtable=True))
    f.write("\\end{table}\n\n")
    #seconda tabella: numero di variabili binarie

    results=[]
    for i, df in enumerate(dfsF):
        column = df["Binary"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of binary variable of Mercedes instances with all path fixed", label="table:mercedes:binaryVar:fixed", longtable=True))
    f.write("\\end{table}\n\n")

    #terza tabella: numero di variabili intere

    results=[]
    for i, df in enumerate(dfsF):
        column = df["Integer"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of integer variable of Mercedes instances with all path fixed", label="table:mercedes:integerVar:fixed", longtable=True))
    f.write("\\end{table}\n\n")
        
    #quarta tabella: numero di variabili lineari

    results=[]
    for i, df in enumerate(dfsF):
        column = df["Linear"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of linear variable of Mercedes instances with all path fixed", label="table:mercedes:linearVar:fixed", longtable=True))
    f.write("\\end{table}\n\n")

    f.write("\\end{document}")


In [19]:
dfsNF = [df_air11NF,df_air10NF,df_air01NF,df_air02NF,df_air03NF,df_grid11NF,df_grid10NF,df_grid01NF,df_grid02NF,df_grid03NF,df_mp11NF,df_mp10NF,df_mp01NF,df_mp02NF,df_mp03NF]
dfsNF += [df_airAP0NF,df_airAP1NF,df_airAP12F,df_gridAP0NF,df_gridAP1NF,df_gridAP2NF,df_mpAP0NF,df_mpAP1NF,df_mpAP2NF,df_airNC0NF,df_airNC1NF,df_airNC2NF,df_gridNC0NF,df_gridNC1NF,df_gridNC2NF,df_mpNC0NF,df_mpNC1NF,df_mpNC2NF]

# Creiamo un file .tex e scriviamo le tabelle con label e caption
with open("/home/magi/UAMdeconflictionMasterThesis/risultati/mercedes/SummaryTablesFree.tex", "w") as f:
    f.write("\\begin{document}\n")
    #prima tabella: numero di variabili totale
    results = []
    for i, df in enumerate(dfsNF):
        column = df["Total"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD": column.std(),
            "Min": column.min(),
            "Max": column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of variable of Mercedes instances with some free path", label="table:mercedes:totalVar:free", longtable=True))
    f.write("\\end{table}\n\n")
    #seconda tabella: numero di variabili binarie

    results=[]
    for i, df in enumerate(dfsNF):
        column = df["Binary"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of binary variable of Mercedes instances with some free path", label="table:mercedes:binaryVar:free", longtable=True))
    f.write("\\end{table}\n\n")

    #terza tabella: numero di variabili intere

    results=[]
    for i, df in enumerate(dfsNF):
        column = df["Integer"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of integer variable of Mercedes instances with some path free", label="table:mercedes:integerVar:free", longtable=True))
    f.write("\\end{table}\n\n")
        
    #quarta tabella: numero di variabili lineari

    results=[]
    for i, df in enumerate(dfsNF):
        column = df["Linear"]
        stats = {
            "DataFrame": get_formatted_name(df["Data Name"].iloc[0]),
            "Mean": column.mean(),
            "STD":  column.std(),
            "Min":  column.min(),
            "Max":  column.max()
        }
        results.append(stats)

    f.write("\\begin{table}[h]\n\\centering\n")
    f.write(pd.DataFrame(results).to_latex(index=False, caption="Statistic about total number of linear variable of Mercedes instances with some free path", label="table:mercedes:linearVar:free", longtable=True))
    f.write("\\end{table}\n\n")

    f.write("\\end{document}")
